In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./dset', train=True,
                                        download=True, transform=transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
model = Model().to(device)

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.166
[1,  4000] loss: 1.791
[1,  6000] loss: 1.660
[1,  8000] loss: 1.575
[1, 10000] loss: 1.530
[1, 12000] loss: 1.467
[2,  2000] loss: 1.416
[2,  4000] loss: 1.383
[2,  6000] loss: 1.370
[2,  8000] loss: 1.334
[2, 10000] loss: 1.333
[2, 12000] loss: 1.301
Finished Training


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layer 1:
        # Conv In: (batch), 3, 32, 32
        # Conv Out: (batch), 32, 32, 32
        # ReLU: (batch), 32, 32, 32
        # MaxPool: (batch), 32, 16, 16
        self.layer1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Layer 2:
        # Conv In: (batch), 32, 16, 16
        # Conv Out: (batch), 64, 16, 16
        # ReLU: (batch), 64, 16, 16
        # MaxPool: (batch), 64, 8, 8
        self.layer2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc = nn.Linear(64 * 8 * 8, 10)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [6]:
device = torch.device("mps")

In [7]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(15):
    for X, Y in trainloader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, Y)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch + 1}/15], Loss: {loss.item():.4f}')

Epoch [1/15], Loss: 0.2472
Epoch [2/15], Loss: 1.9872
Epoch [3/15], Loss: 0.2225
Epoch [4/15], Loss: 0.0388
Epoch [5/15], Loss: 0.3230
Epoch [6/15], Loss: 0.8708
Epoch [7/15], Loss: 0.3608
Epoch [8/15], Loss: 0.6446
Epoch [9/15], Loss: 0.2524
Epoch [10/15], Loss: 0.4133
Epoch [11/15], Loss: 0.1970
Epoch [12/15], Loss: 0.0951
Epoch [13/15], Loss: 0.0452
Epoch [14/15], Loss: 0.3831
Epoch [15/15], Loss: 0.0015


In [10]:
accuracy = 0
with torch.no_grad():
    for X_test, Y_test in trainloader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy += correct_prediction.float().sum()
    accuracy = accuracy.item() / len(trainset)
print('Accuracy: ', accuracy)

Accuracy:  0.8771
